## 1. Parsing gene bank files

### Read GenBank
There are different option to open a genbank file and parse it. One can access to almost every feature and printed in a nice way. It is also possible to save the output in a file. We are going to use as an input the GB file that get got from our blastp search.

In [9]:
from Bio import GenBank
#Replace 'name-file' with the GB file that you got from the blastp run
outputblast = 'output-blastp_gb.txt'

We are going to extract the record ID and the organism name.

In [10]:
#Save in file, 'a' is an option to append and creates file if not existing
f=open('informationFromBlastp.txt', 'a')#Create new file
with open(outputblast) as handle:
    for record in GenBank.parse(handle):
        info = [record. organism, "\t", record.version, '\n']
        f.writelines(info)

f.close()

Once we have the list pf organims with the proteinID, we are going to use it to access entrez

## 2. Accesing entrez using BioPython

In [11]:
import matplotlib.pyplot as plt
import numpy as np

#Pandas for data manipulation
import pandas as pd

from Bio import Entrez
from Bio import SeqIO

Go in ubuntu shell and extract unique organism names

cat output_blastpGBfile.gb | grep -e 'ORGANISM' | cut -d 'M' -f2 > organism_blastp100.txt

### Access genID from proteinID

We are gonna use the list of proteinID and organims that we fetch from the gb file from the blastp output. We will use the proteinID number to match in between the protein database and the nuccore database. The nuccore database has the position in the genome to check in our refseq files (GCF)

In [15]:
fromOutput = pd.read_csv('informationFromBlastp.txt', sep = '\t', header = None)
fromOutput.columns = ["Organism", 'proteinID']
fromOutput.head()

,Organism,proteinID
0,Streptomyces celluloflavus,WP_110952043.1
1,Planktothrix rubescens,ACG63859.1


In [8]:
#fromOutput.iloc[0:10,1]

#### Get all at once and save

In [18]:
#All at once 
#Save assemblies downloaded
save = []
Entrez.email = "dgarcia@eng.au.dk"
for i in fromOutput.iloc[:,1]:
    filename = i.split('.')
    #print(filename[0])
    handle = Entrez.elink(dbfrom = "protein", db = "nuccore", id = i)
    try:
        record = Entrez.read(handle)
        #print(record)
        #Fetch nuccore id
        if len(record[0]["LinkSetDb"]) == 2:
            net_handle = Entrez.efetch(db = "nuccore", id = record[0]["LinkSetDb"][0]["Link"][0]["Id"], rettype = "gb", retmode = "text")
            out_handle = open(filename[0]+'.gb', "w")
            out_handle.write(net_handle.read())
            out_handle.close()
            net_handle.close()
            #Save genbank files found
            save.append(i)
            #print can be removed if wanted
            print("Save", i)
        handle.close()
    except:
        pass


('Save', 'WP_110952043.1')


In [22]:
genbankfilesSaved = pd.Series(save)
genbankfilesSaved.to_csv('genbankfilesSaved.txt', sep ='\t', index = False)

C:\Users\au648169\AppData\Local\Continuum\anaconda2\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


We have extracted all the genbank files from the different proteins identified in the blastp search we will use the genbank files to extract the required ID GCF assemblies that we are going to download using bash scripts.

## 3. Table of GCF ir GCA-ids

Get the overview table where organism names are linked with their GCF id. This is needed to extract the correct genomic fragment to use as an input in antiSmash.

In [ ]:
#Read file generated from running line from downloadingGenomes.sh
fromGCF = pd.read_csv('referenceGenome.txt', sep = '\t', header = None)
#fromGCF.columns = ["GCF_id", 'Bioproject', 'Organism']
fromGCF.columns = ["GCF_id", 'Organism']
fromGCF.head(10)

Merge both documents

In [ ]:
#Merge
GCFidsToUse = pd.merge(fromOutput, fromGCF, on = 'Organism', how = 'outer')
GCFidsToUse = GCFidsToUse.loc[:,['Organism', 'proteinID', 'GCF_id']]
GCFidsToUse.to_csv('GCFids_organism_protein.txt', sep = '\t')